# Two Tunable Qubits + One Tunable Coupler: Engtangling gates

### Imports

In [1]:
import os
import tempfile
import numpy as np
import copy
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.eager.context import num_gpus
import tensorflow_probability as tfp
# Main C3 objects
from c3.c3objs import Quantity as Qty
from c3.parametermap import ParameterMap as PMap
from c3.experiment import Experiment as Exp
from c3.model import Model as Mdl
from c3.generator.generator import Generator as Gnr

# Building blocks
import c3.generator.devices as devices
import c3.signal.gates as gates
import c3.libraries.chip as chip
import c3.signal.pulse as pulse
import c3.libraries.tasks as tasks

# Libs and helpers
import c3.libraries.algorithms as algorithms
import c3.libraries.hamiltonians as hamiltonians
import c3.libraries.fidelities as fidelities
import c3.libraries.envelopes as envelopes
import c3.utils.qt_utils as qt_utils
import c3.utils.tf_utils as tf_utils
from c3.optimizers.optimalcontrol import OptimalControl

#%matplotlib widget
import plotly.graph_objects as go
from utilities_functions import *
from plotting import *

### Define Qubits and coupler

Define Qubits, coupler and couplings

In [3]:
qubit_levels = [3, 3]
qubit_frequencies = [4.16e9, 4.0e9]
anharmonicities = [-220e6, -210e6]
t1 = [60e-6, 30e-6]
t2star = [66e-6, 5e-6]
qubit_temp = [50e-3, 50e-3]
transmon_phi = [1.2088, 0.0]
transmon_phi_0 = [10.0, 10.0]
transmon_d = [0.0, 0.0]


qubits = createQubits(qubit_levels, qubit_frequencies, anharmonicities, t1,
                      t2star, transmon_phi, transmon_phi_0, transmon_d, qubit_temp)

coupler_levels = 3
frequency_coupler = 5.45e9
anharmonicity_coupler = -90e6
t1_coupler = 10e-6
t2star_coupler = 1e-6
coupler_temp = 50e-3
phi_coupler = 3.0227
phi_0_coupler = 10.0
d_coupler = 0.0

c1 = chip.Transmon(
    name="C1",
    desc="Tunable coupler coupling Qubit 1 and Qubit 2",
    freq=Qty(value=frequency_coupler, min_val=2e9, max_val=8e9, unit="Hz 2pi"),
    anhar=Qty(value=anharmonicity_coupler, min_val=-380e6, max_val=-50e6, unit="Hz 2pi"),
    hilbert_dim=coupler_levels,
    t1=Qty(value=t1_coupler, min_val=1e-6, max_val=90e-6, unit="s"),
    t2star=Qty(value=t2star_coupler, min_val=1e-6, max_val=90e-6, unit="s"),
    temp=Qty(value=coupler_temp, min_val=0.0, max_val=0.12, unit="K"),
    phi=Qty(value=phi_coupler, max_val=5.0, min_val=0.0, unit="Wb"),
    phi_0=Qty(value=phi_0_coupler, max_val=11.0, min_val=9.0, unit="Wb"),
    d=Qty(value=d_coupler, max_val=0.1, min_val=-0.1, unit="")
)

# Couple the qubits to the coupler
coupling_strengths = [72.5e6, 71.5e6]
couplings = createChainCouplingsWithCouplers(coupling_strengths, qubits, [c1])

Define the drives (Z-drives are tuning the flux of the qubits)

In [ ]:
drives = createDrives(qubits)

drive_coupler = chip.Drive(
                name = "dC",
                desc= "Drive coupler",
                comment = "Drive for coupler frequency tuning",
                connected = ["C1"],
                hamiltonian_func = hamiltonians.z_drive
)

drive_q1_flux = chip.Drive(
                name = "d1F",
                desc= "Drive qubit 1 flux",
                comment = "Drive for qubit 1 frequency tuning",
                connected = ["Q1"],
                hamiltonian_func = hamiltonians.z_drive
)

drive_q2_flux = chip.Drive(
                name = "d2F",
                desc= "Drive qubit 2 flux",
                comment = "Drive for qubit 2 frequency tuning",
                connected = ["Q2"],
                hamiltonian_func = hamiltonians.z_drive